In [32]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import os

_____

In [33]:
def create_dir(root_dir, path, dest):
    try:
        os.mkdir(root_dir) 
    except OSError as error: 
        pass

    try:
        os.mkdir(path)
    except OSError as error: 
        pass 
    
    try:
        os.mkdir(dest)
    except OSError as error: 
        pass 
        

In [34]:
root_dir = "/your/path/counterfactuals"
directory = f'{root_dir}/explanations'

original_class = ["MB", "EP", "PA", "BG"]
desired_class = ["MB", "EP", "PA", "BG"]

for original in original_class:
    for desired in desired_class:

        merged_data = pd.DataFrame()  # Initialize an empty DataFrame to merge the data
        merged_original = pd.DataFrame()  # Initialize an empty DataFrame to merge the data
        merged_cfes = pd.DataFrame()  # Initialize an empty DataFrame to merge the data

        path = f'{root_dir}/formatted_cfe_from_json'
        output_directory = f'{path}/{original}'
        create_dir(root_dir, path, output_directory)
        # Iterate over the files in the directory
        for filename in os.listdir(directory):
            if filename.endswith(f'{original}_to_{desired}.json'):  # Check if the file has a JSON extension and ends with 'to_MB.json'
                filepath = os.path.join(directory, filename)
                
                # Read the JSON data from file
                with open(filepath, 'r') as file:
                    data = json.load(file)

                # Get original data
                original_df = pd.DataFrame(data['test_data'][0], columns=data['feature_names_including_target'])

                # Get cfes
                cfe_df = pd.DataFrame(data['cfs_list'][0], columns=data['feature_names_including_target'])

                # Merge data into a single DataFrame
                merged_data = pd.concat([merged_data, original_df, cfe_df], ignore_index=True)
                merged_original = pd.concat([merged_original, original_df], ignore_index=True)
                merged_cfes = pd.concat([merged_cfes, cfe_df], ignore_index=True)

        # Save merged data as an Excel file
        merged_data.to_excel(os.path.join(output_directory, f"{original}_to_{desired}_merged_data.xlsx"), index=False)
        merged_original.to_excel(os.path.join(output_directory, f"{original}_to_{desired}_merged_original.xlsx"), index=False)
        merged_cfes.to_excel(os.path.join(output_directory, f"{original}_to_{desired}_merged_cfes.xlsx"), index=False)